MERGE IMDb movies and new_data for plot information


In [37]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist

keywords = pd.read_csv(r"C:\Users\alper\Desktop\IMDb movies.csv", delimiter=',', encoding='utf-8', low_memory=False)
df = pd.read_csv(r"C:\Users\alper\PCSC\Movie-Recommendation-Engine\engine\new_data.csv", delimiter=',', encoding='utf-8', low_memory=False)


# İki DataFrame'deki ortak verilerin bulunması
common_ids = pd.Series(list(set(keywords['imdb_title_id']).intersection(set(df['tconst']))))

# Ortak verilere göre yeni sütun ekleme
merged_data = df[df['tconst'].isin(common_ids)]
merged_data['description'] = keywords[keywords['imdb_title_id'].isin(common_ids)]['description']

nltk.download('stopwords')

def nltkchange(text):
    clean_text = re.sub(r'[^a-zA-ZğüşıöçĞÜŞİÖÇ\s]', '', text)
    txt = clean_text.lower()
    wordlist = txt.split()
    english_stopwords = set(stopwords.words('english'))
    filtered_words = [word for word in wordlist if word not in english_stopwords]
    word_freq = FreqDist(filtered_words)
    most_common_words = [word for word, freq in word_freq.most_common(8)]  # Sadece kelimeleri alıyoruz
    if len(most_common_words) < 1 :
        # DELETED MANUALLY
        return "AAAAAA"
    return ' '.join(most_common_words)

merged_data.dropna(inplace=True)
merged_data['description'] = merged_data['description'].apply(lambda x : nltkchange(x) )

merged_data.to_csv('new_data_withdescription_sum.csv', index=False)


C:\Users\alper\AppData\Local\Temp\ipykernel_36388\174670024.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['description'] = keywords[keywords['imdb_title_id'].isin(common_ids)]['description']
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alper\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\alper\AppData\Local\Temp\ipykernel_36388\174670024.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data.dropna(inplace=True)
C:\Users\alper\AppData\Local\Temp\ipykernel_36388\174

In [47]:
import pandas as pd
import json
from tqdm import tqdm

df = pd.read_csv(r"C:\Users\alper\PCSC\Movie-Recommendation-Engine\keywords.csv", delimiter=',', encoding='utf-8', low_memory=False)
existing_words = set()

unique = {}
i = 1
for index,row in tqdm(df.iterrows(),total=len(df)):
    wordlist = row['description']
    wordlist = wordlist.split()
    for word in wordlist:
        if word not in existing_words:
            unique[i]=word
            existing_words.add(word)
            i += 1
        
print(len(existing_words))

with open('unique_small.json','w') as json_file:
    json.dump(unique,json_file)

100%|██████████| 25797/25797 [00:00<00:00, 72332.36it/s]

28750


In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer

# Veri setinizden bir film konusu örneği (örneğin)
film_konusu = "Skirt-chasing SIC agent Craig Gamble and millionaire bachelor Todd Armstrong set out to foil mad scientist Dr. Goldfoot's plot to use his army of bikini-clad robots to seduce wealthy men into signing over their assets."

# Küçük harfe dönüştürme
film_konusu = film_konusu.lower()

# Cümleyi kelimelere ayırma
words = film_konusu.split()

# Stop words'leri yükleme (gerektiğinde)
nltk.download('stopwords')
nltk.download('')

# Türkçe stop words listesi
turkish_stopwords = set(stopwords.words('english'))

# Stop words'leri cümleden çıkarma
filtered_words = [word for word in words if word not in turkish_stopwords]
# Kelime frekanslarını hesaplama
word_freq = FreqDist(filtered_words)

# En çok geçen kelimeleri ve frekanslarını gösterme
print(word_freq.most_common(8))  # İlk 10 kelimeyi gösteriyoruz, istediğiniz sayıda ayarlayabilirsiniz


[('skirt-chasing', 1), ('sic', 1), ('agent', 1), ('craig', 1), ('gamble', 1), ('millionaire', 1), ('bachelor', 1), ('todd', 1)]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alper\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading : Package '' not found in index


In [22]:
import pandas as pd
from tqdm import tqdm
from engine.Dictionary.classify import keywordIDs
local_movies = pd.read_csv(r'C:\Users\alper\PCSC\Movie-Recommendation-Engine\key_ids.csv',low_memory=False)
movieID = []
words = {'2112', '837' ,'1170','76' ,'27255' ,'2268', '19718' ,'10468' ,'1326', '737', '6973', '1040', '1287', '1119', '536', '6383'}

all_keywords = set(words)
'''   PERFORMANS SORUNU        '''
for index,row in tqdm(local_movies.iterrows(),total=len(local_movies)):
    # OPTIMIZASYON SORUNU
    movie_words =set(row['keyID'].split())
    common_words = movie_words.intersection(all_keywords)
    
    # Ortak kelime varsa, movieID listesine ekleyin
    if common_words:
        movieID.append((row['tconst'], len(common_words)))
        
sorted_movies = sorted(movieID, key=lambda x: x[1],reverse=True)
print(sorted_movies)


  0%|          | 0/25797 [00:00<?, ?it/s]

100%|██████████| 25797/25797 [00:00<00:00, 70313.18it/s]

[('tt10423074', 8), ('tt1225698', 4), ('tt0085387', 3), ('tt0116791', 3), ('tt2076176', 3), ('tt0213802', 3), ('tt0071647', 3), ('tt2368920', 3), ('tt0093067', 3), ('tt2040245', 3), ('tt3343350', 3), ('tt0405024', 3), ('tt0114095', 2), ('tt0250478', 2), ('tt0133122', 2), ('tt2041510', 2), ('tt0075936', 2), ('tt3563156', 2), ('tt0445953', 2), ('tt4466490', 2), ('tt1373120', 2), ('tt0093091', 2), ('tt0087650', 2), ('tt1371574', 2), ('tt0990413', 2), ('tt0077539', 2), ('tt5584796', 2), ('tt0067924', 2), ('tt0091859', 2), ('tt0343221', 2), ('tt0116692', 2), ('tt2872724', 2), ('tt0079322', 2), ('tt0323807', 2), ('tt0084597', 2), ('tt0083675', 2), ('tt0107247', 2), ('tt0082818', 2), ('tt5157326', 2), ('tt3547682', 2), ('tt0216584', 2), ('tt2457282', 2), ('tt6747818', 2), ('tt0118851', 2), ('tt10549312', 2), ('tt0118586', 2), ('tt0118678', 2), ('tt0294252', 2), ('tt1244666', 2), ('tt0201840', 2), ('tt1125929', 2), ('tt4728582', 2), ('tt10264340', 2), ('tt1603362', 2), ('tt0253377', 2), ('tt14

In [8]:
import pandas as pd
from engine.Dictionary.classify import keywordIDs
df = pd.read_csv(r'C:\Users\alper\PCSC\Movie-Recommendation-Engine\keywords.csv',low_memory=False)

def getkeys(text):
    ids = keywordIDs(text)
    idlist =" ".join(ids)
    return idlist

df = df[['tconst','description']]

df['keyID'] = df['description'].apply(lambda x: getkeys(x))

df = df[['tconst','keyID']]

df.to_csv('key_ids.csv', index=False)
